In [1]:
from typing import List, OrderedDict
import os

import flwr
import numpy as np
import torch
from ultralytics import YOLO

In [2]:
def set_parameters(model, parameters: List[np.ndarray]):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

In [3]:
def get_parameters(model) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

In [4]:
class YOLOClient(flwr.client.NumPyClient):
    def __init__(self, model, dataset):
        self.model = model
        self.config_path = f"datasets/{dataset}/config.yaml"
        self.train_size = len(os.listdir(f"datasets/{dataset}/images/train"))

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        self.model.train(data=self.config_path, epochs=1)
        return get_parameters(self.model), self.train_size, {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        metrics = self.model.val()
        loss = metrics.box.loss
        accuracy = metrics.results_dict.get("metrics/precision(B)")
        return loss, self.train_size, {"accuracy": accuracy}

In [5]:
datasets = ["citypersons", "deepdrive", "kitti", "roadsigns"]

def client_fn(cid: str) -> YOLOClient:
    model = YOLO(model="runs/detect/train/weights/best.pt")
    dataset = datasets[int(cid)]
    return YOLOClient(model, dataset).to_client()

In [6]:
NUM_CLIENTS = 4
BATCH_SIZE = 32

In [7]:
strategy = flwr.server.strategy.FedAvg()
client_resources = {"num_cpus": 1, "num_gpus": 1.0}

flwr.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=flwr.server.ServerConfig(num_rounds=1),
    strategy=strategy,
    client_resources=client_resources,
)

INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-06-16 19:23:45,332	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'memory': 7702351872.0, 'node:192.168.2.120': 1.0, 'CPU': 12.0, 'accelerator_type:G': 1.0, 'GPU': 1.0, 'object_store_memory': 3851175936.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


(ClientAppActor pid=19587) Ultralytics YOLOv8.2.32 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)
(ClientAppActor pid=19587) engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/best.pt, data=datasets/roadsigns/config.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /home/pierre/git/newEdge/datasets/roadsigns/labels/train.cache... 693 images, 0 backgrounds, 0 corrupt: 100%|██████████| 693/693 [00:00<?, ?it/s]
val: Scanning /home/pierre/git/newEdge/datasets/roadsigns/labels/val.cache... 184 images, 0 backgrounds, 0 corrupt: 100%|██████████| 184/184 [00:00<?, ?it/s]


(ClientAppActor pid=19587) Plotting labels to runs/detect/train5/labels.jpg... 


  0%|          | 0/44 [00:00<?, ?it/s]


...

ERROR :     Traceback (most recent call last):
  File "/home/pierre/venvs/flwr/lib/python3.11/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 73, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pierre/venvs/flwr/lib/python3.11/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 399, in get_client_result
    return self._fetch_future_result(cid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pierre/venvs/flwr/lib/python3.11/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 280, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
                                         ^^^^^^^^
  File "/home/pierre/venvs/flwr/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/pierre/venvs/flwr/lib

(ClientAppActor pid=19587) Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 1.3ms postprocess per image
(ClientAppActor pid=19587) Results saved to runs/detect/val3
(ClientAppActor pid=19587) Ultralytics YOLOv8.2.32 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5919MiB)


val: Scanning /home/pierre/git/newEdge/datasets/citypersons/labels/val.cache... 500 images, 61 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


(ClientAppActor pid=19587) Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   3%|▎         | 1/32 [00:01<00:50,  1.64s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   6%|▋         | 2/32 [00:01<00:22,  1.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   9%|▉         | 3/32 [00:01<00:14,  1.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  12%|█▎        | 4/32 [00:02<00:11,  2.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  16%|█▌        | 5/32 [00:02<00:09,  2.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  19%|█▉        | 6/32 [00:02<00:09,  2.73it/s]
                 Class     Images  Instances     